In [0]:
if 0:
    # download LJSpeech dataset
    !wget http://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
    # decompress
    !tar -xvjf LJSpeech-1.1.tar.bz2


In [2]:
# This cell imports the drive library and mounts your Google Drive as a VM local drive.  
# You can access to your Drive files using this path "/content/gdrive/My Drive/"

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!pip install ffmpeg
!pip install torchaudio

In [0]:
# lädt mp3 file und konvertiert es zu wav (mit passender sampling rate)

!ffmpeg -i "/content/gdrive/My Drive/text2speech/DieRachederSithStarWarsEpisode3_ep7_Full length.mp3" -acodec pcm_s16le -ar 22050 sample.wav

In [0]:
!mkdir /content/audio

In [6]:
cd /content/audio


/content/audio


In [0]:
# erzeugt mp3 Schnipsel mit 8 sec Dauer
!ffmpeg -i "/content/sample.wav" -f segment -segment_time 4 -c copy out%03d.wav


In [8]:
cd /content

/content


In [0]:
!git clone https://github.com/NVIDIA/waveglow.git


In [10]:
cd waveglow

/content/waveglow


In [0]:
!git submodule init
!git submodule update

In [0]:
!pip3 install -r /content/waveglow/requirements.txt

In [13]:
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
cd apex
pip install -v --no-cache-dir ./


Writing setup.sh


In [14]:
%%writefile  /content/waveglow/config.json

{
    "train_config": {
        "output_directory": "checkpoints",
        "epochs": 100000,
        "learning_rate": 3e-5,
        "sigma": 1.0,
        "iters_per_checkpoint": 300,
        "batch_size": 2,
        "seed": 1234,
        "checkpoint_path": ""
    },
    "data_config": {
        "training_files":"/content/audio",
        "segment_length": 16000,
        "sampling_rate": 22050,
        "filter_length": 1024,
        "hop_length": 256,
        "win_length": 1024,
        "mel_fmin": 0.0,
        "mel_fmax": 8000.0
    },
    "dist_config": {
        "dist_backend": "nccl",
        "dist_url": "tcp://localhost:54321"
    },

    "waveglow_config": {
        "n_mel_channels": 80,
        "n_flows": 12,
        "n_group": 8,
        "n_early_every": 4,
        "n_early_size": 2,
        "WN_config": {
            "n_layers": 8,
            "n_channels": 512,
            "kernel_size": 3
        }
    }
}


Overwriting /content/waveglow/config.json


In [15]:
!unzip '/content/gdrive/My Drive/text2speech/files.zip'    # unzip pretrained model weights

Archive:  /content/gdrive/My Drive/text2speech/files.zip
  inflating: waveglow_256channels_ljs_v2.pt  


In [16]:
cd /content/waveglow

/content/waveglow


In [17]:
%%writefile /content/waveglow/mel2samp.py

import os
import random
import argparse
import json
import torch
import torch.utils.data
import sys
from scipy.io.wavfile import read
import numpy as np
import librosa
# We're using the audio processing from TacoTron2 to make sure it matches
sys.path.insert(0, 'tacotron2')
from tacotron2.layers import TacotronSTFT

MAX_WAV_VALUE = 32768.0



def files_to_list(audio_folder):
    """
    Takes a text file of filenames and makes a list of filenames
    """
    #with open(filename, encoding='utf-8') as f:
    #    files = f.readlines()

    files = [os.path.join(audio_folder,f.rstrip()) for f in os.listdir(audio_folder)]
    return files

  
def load_wav_to_torch(full_path):
    #sampling_rate, data = read(full_path)
    data, sampling_rate = librosa.load(full_path, sr=None)
    #print(data, sampling_rate)
    #print(data.shape)
    #if data.shape[-1]>1 and len(data.shape)>1:
    #    data = data[:,0]
    return torch.FloatTensor(data.astype(np.float32)), sampling_rate


class Mel2Samp(torch.utils.data.Dataset):
    """
    This is the main class that calculates the spectrogram and returns the
    spectrogram, audio pair.
    """
    def __init__(self, training_files, segment_length, filter_length,
                 hop_length, win_length, sampling_rate, mel_fmin, mel_fmax):
      
        self.audio_files = files_to_list(training_files)   
        
        random.seed(1234)
        random.shuffle(self.audio_files)
        self.stft = TacotronSTFT(filter_length=filter_length,
                                 hop_length=hop_length,
                                 win_length=win_length,
                                 sampling_rate=sampling_rate,
                                 mel_fmin=mel_fmin, mel_fmax=mel_fmax)
        self.segment_length = segment_length
        self.sampling_rate = sampling_rate

    def get_mel(self, audio):
        #audio_norm = audio / MAX_WAV_VALUE
        audio_norm = audio.unsqueeze(0)
        audio_norm = torch.autograd.Variable(audio_norm, requires_grad=False)
        melspec = self.stft.mel_spectrogram(audio_norm)
        melspec = torch.squeeze(melspec, 0)
        
        return melspec

    def __getitem__(self, index):
        # Read audio
        filename = self.audio_files[index]
        audio, sampling_rate = load_wav_to_torch(filename)
       
        filename = self.audio_files[index]
        audio, sampling_rate = load_wav_to_torch(filename)
        
        
        #if sampling_rate != self.sampling_rate:
        #    raise ValueError("{} SR doesn't match target {} SR".format(
        #        sampling_rate, self.sampling_rate))

        # Take segment
        if audio.size(0) >= self.segment_length:
            
            max_audio_start = audio.size(0) - self.segment_length
            audio_start = random.randint(0, max_audio_start)
            audio = audio[audio_start:audio_start+self.segment_length]
        else:
            audio = torch.nn.functional.pad(audio, (0, self.segment_length - audio.size(0)), 'constant').data
        mel = self.get_mel(audio)
        
        #audio = audio / MAX_WAV_VALUE
       
        return (mel, audio)

    def __len__(self):
        return len(self.audio_files)



Overwriting /content/waveglow/mel2samp.py


In [0]:
if 0: #visualization of train input
    import matplotlib.pyplot as plt

    mel2samp = Mel2Samp(**{
        "training_files":"/content/audio",
        "segment_length": 16000,
        "sampling_rate": 22050,
        "filter_length": 1024,
        "hop_length": 256,
        "win_length": 1024,
        "mel_fmin": 0.0,
        "mel_fmax": 8000.0
    })
    k=15
    a,b = mel2samp.__getitem__(k)[0],mel2samp.__getitem__(k+1)[0]  
    plt.imshow(a.cpu().numpy())
    plt.show()
    plt.imshow(b.cpu().numpy())
    plt.show()
    a,b =  mel2samp.__getitem__(k)[1],mel2samp.__getitem__(k+1)[1]
    print(a.mean(),a.std(),a.shape,a.min(),b.mean(),b.std(),b.shape,b.max(),b.min())
   

In [0]:
import os
import sys
import time
import subprocess
import argparse

import torch
import torch.distributed as dist
from torch.autograd import Variable

def reduce_tensor(tensor, num_gpus):
    rt = tensor.clone()
    dist.all_reduce(rt, op=dist.reduce_op.SUM)
    rt /= num_gpus
    return rt

def init_distributed(rank, num_gpus, group_name, dist_backend, dist_url):
    assert torch.cuda.is_available(), "Distributed mode requires CUDA."
    print("Initializing Distributed")

    # Set cuda device so everything is done on the right GPU.
    torch.cuda.set_device(rank % torch.cuda.device_count())

    # Initialize distributed communication
    dist.init_process_group(dist_backend, init_method=dist_url,
                            world_size=num_gpus, rank=rank,
                            group_name=group_name)

def _flatten_dense_tensors(tensors):
    """Flatten dense tensors into a contiguous 1D buffer. Assume tensors are of
    same dense type.
    Since inputs are dense, the resulting tensor will be a concatenated 1D
    buffer. Element-wise operation on this buffer will be equivalent to
    operating individually.
    Arguments:
        tensors (Iterable[Tensor]): dense tensors to flatten.
    Returns:
        A contiguous 1D buffer containing input tensors.
    """
    if len(tensors) == 1:
        return tensors[0].contiguous().view(-1)
    flat = torch.cat([t.contiguous().view(-1) for t in tensors], dim=0)
    return flat

def _unflatten_dense_tensors(flat, tensors):
    """View a flat buffer using the sizes of tensors. Assume that tensors are of
    same dense type, and that flat is given by _flatten_dense_tensors.
    Arguments:
        flat (Tensor): flattened dense tensors to unflatten.
        tensors (Iterable[Tensor]): dense tensors whose sizes will be used to
          unflatten flat.
    Returns:
        Unflattened dense tensors with sizes same as tensors and values from
        flat.
    """
    outputs = []
    offset = 0
    for tensor in tensors:
        numel = tensor.numel()
        outputs.append(flat.narrow(0, offset, numel).view_as(tensor))
        offset += numel
    return tuple(outputs)

def apply_gradient_allreduce(module):
    """
    Modifies existing model to do gradient allreduce, but doesn't change class
    so you don't need "module"
    """
    if not hasattr(dist, '_backend'):
        module.warn_on_half = True
    else:
        module.warn_on_half = True if dist._backend == dist.dist_backend.GLOO else False

    for p in module.state_dict().values():
        if not torch.is_tensor(p):
            continue
        dist.broadcast(p, 0)

    def allreduce_params():
        if(module.needs_reduction):
            module.needs_reduction = False
            buckets = {}
            for param in module.parameters():
                if param.requires_grad and param.grad is not None:
                    tp = type(param.data)
                    if tp not in buckets:
                        buckets[tp] = []
                    buckets[tp].append(param)
            if module.warn_on_half:
                if torch.cuda.HalfTensor in buckets:
                    print("WARNING: gloo dist backend for half parameters may be extremely slow." +
                          " It is recommended to use the NCCL backend in this case. This currently requires" +
                          "PyTorch built from top of tree master.")
                    module.warn_on_half = False

            for tp in buckets:
                bucket = buckets[tp]
                grads = [param.grad.data for param in bucket]
                coalesced = _flatten_dense_tensors(grads)
                dist.all_reduce(coalesced)
                coalesced /= dist.get_world_size()
                for buf, synced in zip(grads, _unflatten_dense_tensors(coalesced, grads)):
                    buf.copy_(synced)

    for param in list(module.parameters()):
        def allreduce_hook(*unused):
            Variable._execution_engine.queue_callback(allreduce_params)
        if param.requires_grad:
            param.register_hook(allreduce_hook)
            dir(param)

    def set_needs_reduction(self, input, output):
        self.needs_reduction = True

    module.register_forward_hook(set_needs_reduction)
    return module


def main(config, stdout_dir, args_str):
    args_list = ['train.py']
    args_list += args_str.split(' ') if len(args_str) > 0 else []

    args_list.append('--config={}'.format(config))

    num_gpus = torch.cuda.device_count()
    args_list.append('--num_gpus={}'.format(num_gpus))
    args_list.append("--group_name=group_{}".format(time.strftime("%Y_%m_%d-%H%M%S")))

    if not os.path.isdir(stdout_dir):
        os.makedirs(stdout_dir)
        os.chmod(stdout_dir, 0o775)

    workers = []

    for i in range(num_gpus):
        args_list[-2] = '--rank={}'.format(i)
        stdout = None if i == 0 else open(
            os.path.join(stdout_dir, "GPU_{}.log".format(i)), "w")
        print(args_list)
        p = subprocess.Popen([str(sys.executable)]+args_list, stdout=stdout)
        workers.append(p)

    for p in workers:
        p.wait()


In [0]:

model_path = '/content/gdrive/My Drive/text2speech/waveglow_256channels.pt'  # path of pretrained model (optional)
out_check_path = '/content/gdrive/My Drive/text2speech/ceckpoint_out_1.pth'  # path where model, optimzer,... are saved
in_check_path = '/content/gdrive/My Drive/text2speech/ceckpoint_out.pth' # path of path of previous checkpoint
load_check = 1  # 1 if previous checkpoint exists, otherwise 0
from_scratch = 1 - load_check

In [22]:
cd /content/waveglow

/content/waveglow


In [23]:
import json
import os
import torch
from convert_model import update_model

#=====START: ADDED FOR DISTRIBUTED======
#from distributed import init_distributed, apply_gradient_allreduce, reduce_tensor
from torch.utils.data.distributed import DistributedSampler
#=====END:   ADDED FOR DISTRIBUTED======

from torch.utils.data import DataLoader
from glow import WaveGlow,WaveGlowLoss
from mel2samp import Mel2Samp



def load_checkpoint(checkpoint_path, model, optimizer):
    #assert os.path.isfile(checkpoint_path)
    checkpoint_dict = torch.load(in_check_path, map_location='cpu')
    iteration = checkpoint_dict['iteration']
    optimizer.load_state_dict(checkpoint_dict['optimizer'])
    model_for_loading = checkpoint_dict['model']
    model = checkpoint_dict['model'].cuda()
    #model.load_state_dict(model_for_loading.state_dict())
    print("Loaded checkpoint '{}' (iteration {})" .format(
          in_check_path, iteration))
    return model, optimizer, iteration

def save_checkpoint(model, optimizer, learning_rate, iteration, filepath):
    print("Saving model and optimizer state at iteration {} to {}".format(
          iteration, filepath))
    #model_for_saving = WaveGlow(**waveglow_config).cuda()
    #model_for_saving.load_state_dict(model.state_dict())
    torch.save({'model': model,
                'iteration': iteration,
                'optimizer': optimizer.state_dict(),
                'learning_rate': learning_rate}, filepath)
    


def train(num_gpus, rank, group_name, output_directory, epochs, learning_rate,
          sigma, iters_per_checkpoint, batch_size, seed, checkpoint_path):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    #=====START: ADDED FOR DISTRIBUTED======
    if num_gpus > 1:
        init_distributed(rank, num_gpus, group_name, **dist_config)
    #=====END:   ADDED FOR DISTRIBUTED======

    criterion = WaveGlowLoss(sigma)
    
    
    model = WaveGlow(**waveglow_config).cuda()
    if from_scratch:
        model = torch.load(model_path, map_location='cpu')['model']
        model = update_model(model)
    model.cuda()
   

    #=====START: ADDED FOR DISTRIBUTED======
    if num_gpus > 1:
        model = apply_gradient_allreduce(model)
    #=====END:   ADDED FOR DISTRIBUTED======

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Load checkpoint if one exists
    iteration = 0
    if load_check:
        model, optimizer, iteration = load_checkpoint(checkpoint_path, model,
                                                      optimizer)
        iteration += 1  # next iteration is iteration + 1

    trainset = Mel2Samp(**data_config)
    # =====START: ADDED FOR DISTRIBUTED======
    train_sampler = DistributedSampler(trainset) if num_gpus > 1 else None
    # =====END:   ADDED FOR DISTRIBUTED======
    train_loader = DataLoader(trainset, num_workers=0, shuffle=True,
                              sampler=train_sampler,
                              batch_size=batch_size,
                              pin_memory=False,
                              drop_last=True)

    # Get shared output_directory ready
    if rank == 0:
        if not os.path.isdir(output_directory):
            os.makedirs(output_directory)
            os.chmod(output_directory, 0o775)
        print("output directory", output_directory)

    model.train()
    epoch_offset = max(0, int(iteration / len(train_loader)))
    # ================ MAIN TRAINNIG LOOP! ===================
    
    steps = 4
    avg_loss = 0
    for epoch in range(epoch_offset, epochs):
        print("Epoch: {}".format(epoch))
        for i, batch in enumerate(train_loader):
            model.zero_grad()

            mel, audio = batch
           
            mel = torch.autograd.Variable(mel.cuda())
            audio = torch.autograd.Variable(audio.cuda())
            outputs = model((mel, audio))

            loss = criterion(outputs)/steps
            avg_loss += loss.item()
            loss.backward()
            
            
            if i%steps == 0 and i>0:
                print("train_loss for iteration {}:\t{:.9f}".format(iteration, avg_loss))
                avg_loss = 0
                
                optimizer.step()
                model.zero_grad()
                              
            ###
 
            if (iteration % iters_per_checkpoint == 0):
                if rank == 0:
                    #checkpoint_path = "{}/waveglow_{}".format(
                    #    output_directory, iteration)
                    save_checkpoint(model, optimizer, learning_rate, iteration,out_check_path)

            iteration += 1
            
            
    return model

if __name__ == "__main__":


    # Parse configs.  Globals nicer in this case
    with open('/content/waveglow/config.json') as f:
        data = f.read()
    config = json.loads(data)
    train_config = config["train_config"]
    global data_config
    data_config = config["data_config"]
    global dist_config
    dist_config = config["dist_config"]
    global waveglow_config
    waveglow_config = config["waveglow_config"]



    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = False
    model = train(1, 0,"", **train_config)


Loaded checkpoint '/content/gdrive/My Drive/text2speech/ceckpoint_out.pth' (iteration 5700)
output directory checkpoints
Epoch: 0
train_loss for iteration 5705:	-6.502592206
train_loss for iteration 5709:	-5.631444693
train_loss for iteration 5713:	-5.639689088
train_loss for iteration 5717:	-5.469002008


KeyboardInterrupt: ignored

In [0]:
!unzip '/content/Nbc_mel.zip' -d '/content'

In [40]:
### testing of progress on predicted mel spectrograms
import os
from scipy.io.wavfile import write
import torch
import numpy as np
from mel2samp import files_to_list, MAX_WAV_VALUE
from convert_model import update_model
import pickle
from denoiser import Denoiser


#model_path = '/content/gdrive/My Drive/text2speech/waveglow_256channels.pt'
model_path = out_check_path
sampling_rate = 22050
sigma = 0.8
#denoiser_strength = 0.2


waveglow = torch.load(model_path)['model']
#waveglow = update_model(waveglow)
waveglow = waveglow.remove_weightnorm(waveglow)
waveglow.cuda().eval()

#denoise = Denoiser(waveglow)

path = '/content/mel_preds/'
mel_files = os.listdir(path)
audios = []
for i, file_path in enumerate(mel_files[:2]):
    p = path+file_path
    print(p)

    mel = np.load(p)
    mel = torch.from_numpy(mel)
    if len(mel.size())<3:
        mel = mel[None,...]
    mel = torch.autograd.Variable(mel.cuda())

    with torch.no_grad():
        #mel = mel2samp.__getitem__(i)[0].cuda()[None,...]
        audio = waveglow.infer(mel, sigma=sigma)
        #if denoiser_strength > 0:
        #    audio = denoise(audio.cuda()).cpu()
        audio = audio * MAX_WAV_VALUE
    audio = audio.squeeze()
    audio = audio.cpu().numpy()
    audios+=[audio]



    

/content/mel_preds/189.npy
/content/mel_preds/129.npy


In [31]:
from IPython.display import Audio


Audio(audios[1], rate= 22050)